<p style="text-align: center;">МИНИСТЕРСТВО ОБРАЗОВАНИЯ И НАУКИ РОССИЙСКОЙ ФЕДЕРАЦИИ
<p style="text-align: center;">Федеральное государственное автономное образовательное учреждение высшего образования «Самарский национальный исследовательский университет имени академика С. П. Королева» (Самарский университет)</p>
<br> <br> <br>
<p style="text-align: center;">Институт информатики и кибернетики
<p style="text-align: center;">Факультет информатики
<p style="text-align: center;">Кафедра программных систем
<br><br><br>
<p style="text-align: center;">ОТЧЁТ
<p style="text-align: center;">по лабораторной работе № 2
<p style="text-align: center;">«Введение в Spark с использованием Python (часть 2)»
<p style="text-align: center;">по курсу «Интеллектуальный анализ данных и большие данные»
<p style="text-align: center;">
<br><br><br><br><br><br><br><br>
<p style="text-align: right;">Выполнил: Яшин И.А.
<p style="text-align: right;">гр. 6132-020402D
<p style="text-align: right;">
<br><br><br><br><br><br><br><br><br>
<p style="text-align: center;">Самара 2025

In [ ]:
import os

os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

import numpy as np
import pandas as pd
import pyspark.pandas as ps
from pyspark.sql import SparkSession

In [ ]:
s = ps.Series([1, 3, 5, np.nan, 6, 8])

In [ ]:
psdf = ps.DataFrame(
    {'a': [1, 2, 3, 4, 5, 6],
     'b': [100, 200, 300, 400, 500, 600],
     'c': ["one", "two", "three", "four", "five", "six"]},
    index=[10, 20, 30, 40, 50, 60])
psdf

In [ ]:
dates = pd.date_range('20130101', periods=6)
dates

In [ ]:
pdf = pd.DataFrame(np.random.randn(6, 4), index=dates, columns=list('ABCD'))
pdf

In [ ]:
psdf = ps.from_pandas(pdf)
psdf

In [ ]:
type(psdf)

In [ ]:
spark = SparkSession.builder.getOrCreate()
sdf = spark.createDataFrame(pdf)
sdf.show()

In [ ]:
psdf = sdf.pandas_api()
psdf

In [ ]:
psdf.dtypes

In [ ]:
psdf.head()

In [ ]:
psdf.index

In [ ]:
psdf.columns

In [ ]:
psdf.to_numpy()

In [ ]:
psdf.describe()

In [ ]:
psdf.T

In [ ]:
psdf.sort_index(ascending=False)

In [ ]:
psdf.sort_values(by='B')

In [ ]:
pdf1 = pdf.reindex(index=dates[0:4], columns=list(pdf.columns) + ['E'])
pdf1.loc[dates[0]:dates[1], 'E'] = 1
psdf1 = ps.from_pandas(pdf1)
psdf1

In [ ]:
psdf1.dropna(how='any')

In [ ]:
psdf1.fillna(value=5)

In [ ]:
psdf.mean()

In [ ]:
prev = spark.conf.get("spark.sql.execution.arrow.pyspark.enabled")  # Keep its default value.
ps.set_option("compute.default_index_type", "distributed")  # Use default index prevent overhead.
import warnings
warnings.filterwarnings("ignore")  # Ignore warnings coming from Arrow optimizations.

In [ ]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", True)
%timeit ps.range(300000).to_pandas()

In [ ]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", False)
%timeit ps.range(300000).to_pandas()

In [ ]:
ps.reset_option("compute.default_index_type")
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", prev)  # Set its default value back.

In [ ]:
psdf = ps.DataFrame({'A': ['foo', 'bar', 'foo', 'bar',
                          'foo', 'bar', 'foo', 'foo'],
                    'B': ['one', 'one', 'two', 'three',
                          'two', 'two', 'one', 'three'],
                    'C': np.random.randn(8),
                    'D': np.random.randn(8)})
psdf

In [ ]:
psdf.groupby('A').sum()

In [ ]:
psdf.groupby(['A', 'B']).sum()

In [ ]:
pser = pd.Series(np.random.randn(1000),
                 index=pd.date_range('1/1/2000', periods=1000))
psser = ps.Series(pser)
psser = psser.cummax()
psser.plot()

In [ ]:
pdf = pd.DataFrame(np.random.randn(1000, 4), index=pser.index,
                   columns=['A', 'B', 'C', 'D'])
psdf = ps.from_pandas(pdf)
psdf = psdf.cummax()
psdf.plot()

In [ ]:
psdf.to_csv('foo.csv')
ps.read_csv('foo.csv').head(10)

In [ ]:
psdf.to_parquet('bar.parquet')
ps.read_parquet('bar.parquet').head(10)

In [ ]:
psdf.to_spark_io('zoo.orc', format="orc")
ps.read_spark_io('zoo.orc', format="orc").head(10)